### SQL Databases

In [1]:
import sqlite3 as sq3
import numpy as np
import pandas as pd
import datetime as dt
from pylab import mpl, plt
# import warnings; warnings.simplefilter('ignore')

Python can work with any kind of Structured Query Language (SQL) database, and
in general also with any kind of NoSQL database.   
One SQL or relational database that is delivered with Python by default is SQLite3.   
With it, the basic Python approach to SQL databases can be easily illustrated :  

In [2]:
raw = pd.read_csv('C:/Users/Matthieu/Documents/tr_eikon_eod_data.csv',
                  index_col=0, parse_dates=True).dropna()

In [3]:
path = 'C:/Users/Matthieu/Documents/'

In [4]:
con = sq3.connect(path + 'numbs.db')
# Opens a database connection; a file is created if it does not exist.

In [5]:
query = 'CREATE TABLE numbs2 (Date date, No1 real, No2 real)'  
# A SQL query that creates a table with three columns.

In [6]:
# con.execute(query)  

In [7]:
con.commit()  
#… and commits the changes.

In [8]:
q = con.execute  
# Defines a short alias for the con.execute() method.

In [9]:
q('SELECT * FROM sqlite_master').fetchall()  
# Fetches metainformation about the database, showing the just-created table as the single object.

[('table',
  'numbs1',
  'numbs1',
  4,
  'CREATE TABLE numbs1 (Date date, No1 real, No2 real)'),
 ('table',
  'numbs2',
  'numbs2',
  5,
  'CREATE TABLE numbs2 (Date date, No1 real, No2 real)')]

In [10]:
q('SELECT "numbs2" FROM sqlite_master').fetchall()

[('numbs2',), ('numbs2',)]

Now that there is a database file with a table, this table can be populated with data.
Each row consists of a datetime object and two float objects :

In [11]:
import datetime

In [12]:
now = datetime.datetime.now()
q('INSERT INTO numbs2 VALUES(?, ?, ?)', (now, 0.12, 7.3))  
# Writes a single row (or record) to the numbs table

In [13]:
np.random.seed(100)

In [14]:
data = np.random.standard_normal((10000, 2)).round(4)  
# Creates a larger dummy data set as an ndarray object.

In [15]:
%%time 
for row in data:  
    now = datetime.datetime.now()
    q('INSERT INTO numbs VALUES(?, ?, ?)', (now, row[0], row[1]))
con.commit()
# Iterates over the rows of the ndarray object.

OperationalError: no such table: numbs

In [16]:
q('SELECT * FROM numbs2').fetchmany(4)  
# Retrieves a number of rows from the table.

[('2024-09-16 00:19:16.249641', 0.12, 7.3)]

In [17]:
q('SELECT * FROM numbs2 WHERE no1 > 0.5').fetchmany(4)  
# The same, but with a condition on the values in the No1 column.

[]

In [18]:
pointer = q('SELECT * FROM numbs2')  
# Defines a pointer object …

In [19]:
for i in range(3):
    print(pointer.fetchone())  
# … that behaves like a generator object.

('2024-09-16 00:19:16.249641', 0.12, 7.3)
None
None


In [20]:
rows = pointer.fetchall()  
rows[:3]
# Retrieves all the remaining rows.

[]

Finally, one might want to delete the table object in the database if it’s not required
anymore:

In [22]:
q('DROP TABLE IF EXISTS numbs')  
# Removes the table from the database.
# HERE!!

In [23]:
q('SELECT * FROM sqlite_master').fetchall()  
# There are no table objects left after this operation. ??

[('table',
  'numbs1',
  'numbs1',
  4,
  'CREATE TABLE numbs1 (Date date, No1 real, No2 real)'),
 ('table',
  'numbs2',
  'numbs2',
  5,
  'CREATE TABLE numbs2 (Date date, No1 real, No2 real)')]

In [24]:
con.close()  
# Closes the database connection.

In [25]:
!rm -f $path*  
# Removes the database file from disk. ??

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


### Writing and Reading Numpy Arrays

In [26]:
dtimes = np.arange('2019-01-01 10:00:00', '2025-12-31 22:00:00',
                  dtype='datetime64[m]')  
# Creates an ndarray object with datetime as the dtype.

In [27]:
len(dtimes)

3681360

In [29]:
dty = np.dtype([('Date', 'datetime64[m]'),
                ('No1', 'f'), ('No2', 'f')])  
# Defines the special dtype object for the structured array.

In [30]:
data = np.zeros(len(dtimes), dtype=dty)  
# Instantiates an ndarray object with the special dtype

In [31]:
data['Date'] = dtimes  
# Populates the Date column.

In [32]:
a = np.random.standard_normal((len(dtimes), 2)).round(4)  
# The dummy data sets …

In [40]:
data['No1'] = a[:, 0]  
data['No2'] = a[:, 1]  
# … which populate the No1 and No2 columns.

In [33]:
data.nbytes  
# The size of the structured array in bytes.

58901760

In [ ]:
%time np.save(path + 'array', data)  

In [ ]:
ll $path*  

In [ ]:
%time np.load(path + 'array.npy')  

In [ ]:
%time data = np.random.standard_normal((10000, 6000)).round(4)  

In [ ]:
data.nbytes  

In [ ]:
%time np.save(path + 'array', data)  

In [ ]:
ll $path*   

In [ ]:
%time np.load(path + 'array.npy')  

In [ ]:
!rm -f $path*

## I/O with pandas

In [ ]:
data = np.random.standard_normal((1000000, 5)).round(4)

In [ ]:
data[:3]

### SQL Database

In [ ]:
filename = path + 'numbers'

In [ ]:
con = sq3.Connection(filename + '.db')

In [ ]:
query = 'CREATE TABLE numbers (No1 real, No2 real,\
        No3 real, No4 real, No5 real)'  

In [ ]:
q = con.execute
qm = con.executemany

In [ ]:
q(query)

In [ ]:
%%time
qm('INSERT INTO numbers VALUES (?, ?, ?, ?, ?)', data)  
con.commit()

In [ ]:
ll $path*

In [ ]:
%%time
temp = q('SELECT * FROM numbers').fetchall()  
print(temp[:3])

In [ ]:
%%time
query = 'SELECT * FROM numbers WHERE No1 > 0 AND No2 < 0'
res = np.array(q(query).fetchall()).round(3)  

In [ ]:
res = res[::100]  
plt.figure(figsize=(10, 6))
plt.plot(res[:, 0], res[:, 1], 'ro');

### From SQL to pandas

In [ ]:
%time data = pd.read_sql('SELECT * FROM numbers', con)  

In [ ]:
data.head()

In [ ]:
%time data[(data['No1'] > 0) & (data['No2'] < 0)].head()  

In [ ]:
%%time
q = '(No1 < -0.5 | No1 > 0.5) & (No2 < -1 | No2 > 1)'  
res = data[['No1', 'No2']].query(q)  

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(res['No1'], res['No2'], 'ro');

In [ ]:
h5s = pd.HDFStore(filename + '.h5s', 'w')  

In [ ]:
%time h5s['data'] = data  

In [ ]:
h5s  

In [ ]:
h5s.close()  

In [ ]:
%%time
h5s = pd.HDFStore(filename + '.h5s', 'r')  
data_ = h5s['data']  
h5s.close()  

In [ ]:
data_ is data  

In [ ]:
(data_ == data).all()  

In [ ]:
np.allclose(data_, data)  

In [ ]:
ll $path*  

### Data as CSV File

In [ ]:
%time data.to_csv(filename + '.csv')  

In [ ]:
ll $path

In [ ]:
%time df = pd.read_csv(filename + '.csv')  

In [ ]:
df[['No1', 'No2', 'No3', 'No4']].hist(bins=20, figsize=(10, 6));

### Data as Excel File

In [ ]:
%time data[:100000].to_excel(filename + '.xlsx')

In [ ]:
%time df = pd.read_excel(filename + '.xlsx', 'Sheet1', index_col=0)  

In [ ]:
df.info()

In [ ]:
df.cumsum().plot(figsize=(10, 6));

In [ ]:
ll $path*

In [ ]:
rm -f $path*

## Fast I/O with PyTables

In [ ]:
import tables as tb  
import datetime as dt

### Working with Tables

In [ ]:
filename = path + 'pytab.h5'

In [ ]:
h5 = tb.open_file(filename, 'w')  

In [ ]:
row_des = {
    'Date': tb.StringCol(26, pos=1),  
    'No1': tb.IntCol(pos=2),  
    'No2': tb.IntCol(pos=3),  
    'No3': tb.Float64Col(pos=4),  
    'No4': tb.Float64Col(pos=5)  
    }

In [ ]:
rows = 2000000

In [ ]:
filters = tb.Filters(complevel=0)  

In [ ]:
tab = h5.create_table('/', 'ints_floats',  
                      row_des,  
                      title='Integers and Floats',  
                      expectedrows=rows,  
                      filters=filters)  

In [ ]:
type(tab)

In [ ]:
tab

In [ ]:
pointer = tab.row  

In [ ]:
ran_int = np.random.randint(0, 10000, size=(rows, 2))  

In [ ]:
ran_flo = np.random.standard_normal((rows, 2)).round(4)  

In [ ]:
%%time
for i in range(rows):
    pointer['Date'] = dt.datetime.now()  
    pointer['No1'] = ran_int[i, 0]  
    pointer['No2'] = ran_int[i, 1]  
    pointer['No3'] = ran_flo[i, 0]  
    pointer['No4'] = ran_flo[i, 1]  
    pointer.append()  
tab.flush()  

In [ ]:
tab  

In [ ]:
ll $path*

In [ ]:
dty = np.dtype([('Date', 'S26'), ('No1', '<i4'), ('No2', '<i4'),
                                 ('No3', '<f8'), ('No4', '<f8')])  

In [ ]:
sarray = np.zeros(len(ran_int), dtype=dty)  

In [ ]:
sarray[:4]  

In [ ]:
%%time
sarray['Date'] = dt.datetime.now()  
sarray['No1'] = ran_int[:, 0]  
sarray['No2'] = ran_int[:, 1]  
sarray['No3'] = ran_flo[:, 0]  
sarray['No4'] = ran_flo[:, 1]  

In [ ]:
%%time
h5.create_table('/', 'ints_floats_from_array', sarray,
                      title='Integers and Floats',
                      expectedrows=rows, filters=filters)  

In [ ]:
type(h5)

In [ ]:
h5  

In [ ]:
h5.remove_node('/', 'ints_floats_from_array')  

In [ ]:
tab[:3]  

In [ ]:
tab[:4]['No4']  

In [ ]:
%time np.sum(tab[:]['No3'])  

In [ ]:
%time np.sum(np.sqrt(tab[:]['No1']))  

In [ ]:
%%time
plt.figure(figsize=(10, 6))
plt.hist(tab[:]['No3'], bins=30);

In [ ]:
query = '((No3 < -0.5) | (No3 > 0.5)) & ((No4 < -1) | (No4 > 1))'  

In [ ]:
iterator = tab.where(query)  

In [ ]:
%time res = [(row['No3'], row['No4']) for row in iterator]  

In [ ]:
res = np.array(res)  
res[:3]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(res.T[0], res.T[1], 'ro');

In [ ]:
%%time
values = tab[:]['No3']
print('Max %18.3f' % values.max())
print('Ave %18.3f' % values.mean())
print('Min %18.3f' % values.min())
print('Std %18.3f' % values.std())

In [ ]:
%%time
res = [(row['No1'], row['No2']) for row in
        tab.where('((No1 > 9800) | (No1 < 200)) \
                & ((No2 > 4500) & (No2 < 5500))')]

In [ ]:
for r in res[:4]:
    print(r)

In [ ]:
%%time
res = [(row['No1'], row['No2']) for row in
        tab.where('(No1 == 1234) & (No2 > 9776)')]

In [ ]:
for r in res:
    print(r)

### Working with Compressed Tables

In [ ]:
filename = path + 'pytabc.h5'

In [ ]:
h5c = tb.open_file(filename, 'w') 

In [ ]:
filters = tb.Filters(complevel=5,  
                     complib='blosc')  

In [ ]:
tabc = h5c.create_table('/', 'ints_floats', sarray,
                        title='Integers and Floats',
                        expectedrows=rows, filters=filters)

In [ ]:
query = '((No3 < -0.5) | (No3 > 0.5)) & ((No4 < -1) | (No4 > 1))'

In [ ]:
iteratorc = tabc.where(query)  

In [ ]:
%time res = [(row['No3'], row['No4']) for row in iteratorc]  

In [ ]:
res = np.array(res)
res[:3]

In [ ]:
%time arr_non = tab.read()  

In [ ]:
tab.size_on_disk

In [ ]:
arr_non.nbytes

In [ ]:
%time arr_com = tabc.read()  

In [ ]:
tabc.size_on_disk

In [ ]:
arr_com.nbytes

In [ ]:
ll $path*  

In [ ]:
h5c.close()  

### Working with Arrays

In [ ]:
%%time
arr_int = h5.create_array('/', 'integers', ran_int)  
arr_flo = h5.create_array('/', 'floats', ran_flo)  

In [ ]:
h5  

In [ ]:
ll $path*

In [ ]:
h5.close()

In [ ]:
!rm -f $path*

### Out-of-Memory Computations

In [ ]:
filename = path + 'earray.h5'

In [ ]:
h5 = tb.open_file(filename, 'w') 

In [ ]:
n = 500  

In [ ]:
ear = h5.create_earray('/', 'ear',  
                      atom=tb.Float64Atom(),  
                      shape=(0, n))  

In [ ]:
type(ear)

In [ ]:
rand = np.random.standard_normal((n, n))  
rand[:4, :4]

In [ ]:
%%time
for _ in range(750):
    ear.append(rand)  
ear.flush()

In [ ]:
ear

In [ ]:
ear.size_on_disk

In [ ]:
out = h5.create_earray('/', 'out',
                      atom=tb.Float64Atom(),
                      shape=(0, n))

In [ ]:
out.size_on_disk

In [ ]:
expr = tb.Expr('3 * sin(ear) + sqrt(abs(ear))')  

In [ ]:
expr.set_output(out, append_mode=True)  

In [ ]:
%time expr.eval()  

In [ ]:
out.size_on_disk

In [ ]:
out[0, :10]

In [ ]:
%time out_ = out.read()  

In [ ]:
out_[0, :10]

In [ ]:
import numexpr as ne  

In [ ]:
expr = '3 * sin(out_) + sqrt(abs(out_))'  

In [ ]:
ne.set_num_threads(1)  

In [ ]:
%time ne.evaluate(expr)[0, :10]  

In [ ]:
ne.set_num_threads(4)  

In [ ]:
%time ne.evaluate(expr)[0, :10]  

In [ ]:
h5.close()

In [ ]:
!rm -f $path*

## TsTables

### Sample Data

In [ ]:
no = 5000000  
co = 3  
interval = 1. / (12 * 30 * 24 * 60)  
vol = 0.2  

In [ ]:
%%time
rn = np.random.standard_normal((no, co))  
rn[0] = 0.0  
paths = 100 * np.exp(np.cumsum(-0.5 * vol ** 2 * interval +
        vol * np.sqrt(interval) * rn, axis=0))  
paths[0] = 100  

In [ ]:
dr = pd.date_range('2019-1-1', periods=no, freq='1s')

In [ ]:
dr[-6:]

In [ ]:
df = pd.DataFrame(paths, index=dr, columns=['ts1', 'ts2', 'ts3'])

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df[::100000].plot(figsize=(10, 6));
# plt.savefig('../../images/ch09/io_07.png')

### Data Storage (with `tstables`)

<b style="color: red;">Install the package as follows:</b>

    pip install git+https://github.com/yhilpisch/tstables

In [ ]:
import tstables as tstab

In [ ]:
class ts_desc(tb.IsDescription):
    timestamp = tb.Int64Col(pos=0)  
    ts1 = tb.Float64Col(pos=1)  
    ts2 = tb.Float64Col(pos=2)  
    ts3 = tb.Float64Col(pos=3)  

In [ ]:
h5 = tb.open_file(path + 'tstab.h5', 'w')  

In [ ]:
ts = h5.create_ts('/', 'ts', ts_desc)  

In [ ]:
%time ts.append(df)  

In [ ]:
type(ts)

In [ ]:
ls -n $path

In [ ]:
read_start_dt = dt.datetime(2019, 2, 1, 0, 0)  
read_end_dt = dt.datetime(2019, 2, 5, 23, 59)  #<2>

In [ ]:
%time rows = ts.read_range(read_start_dt, read_end_dt)  

In [ ]:
rows.info()  

In [ ]:
rows.head()  

In [ ]:
h5.close()

In [ ]:
(rows[::500] / rows.iloc[0]).plot(figsize=(10, 6));

In [ ]:
import random

In [ ]:
h5 = tb.open_file(path + 'tstab.h5', 'r')

In [ ]:
ts = h5.root.ts._f_get_timeseries()  

In [ ]:
%%time
for _ in range(100):  
    d = random.randint(1, 24)  
    read_start_dt = dt.datetime(2019, 2, d, 0, 0, 0)
    read_end_dt = dt.datetime(2019, 2, d + 3, 23, 59, 59)
    rows = ts.read_range(read_start_dt, read_end_dt)

In [ ]:
rows.info()  

In [ ]:
!rm $path/tstab.h5

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>